In [6]:
from google.colab import drive
drive.mount('/content/drive')
# Then use this as your output_dir
output_dir = "/content/drive/MyDrive/qlora-finetuned-model"

Mounted at /content/drive


In [7]:
!pip install transformers peft accelerate datasets bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset

In [9]:
model_name = "tiiuae/falcon-rw-1b"
dataset = load_dataset("databricks/databricks-dolly-15k")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [13]:
model = prepare_model_for_kbit_training(model)

In [14]:
# Add LoRA adapters
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],  # change per model
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [15]:
model = get_peft_model(model, peft_config)

In [16]:
def tokenize(example):
    tokens = tokenizer(
        example['instruction'] + "\n" + example['response'],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()  # 👈 add this line
    return tokens

In [17]:
tokenized_data = dataset['train'].map(tokenize)
train_dataset=tokenized_data.select(range(2000))  # Try 1k–2k samples first

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [18]:
model.gradient_checkpointing_enable()

In [19]:
# Training
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=2,
    logging_dir="./logs",
    output_dir="./qlora-finetuned-model",
    save_strategy="steps",
    save_steps=500,
    fp16=True
)

In [20]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    args=training_args
)

<ipython-input-20-3f8470b5f275>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
from transformers.trainer_utils import get_last_checkpoint

output_dir = "./qlora-finetuned-model"
checkpoint = get_last_checkpoint(output_dir)

trainer.train(resume_from_checkpoint=checkpoint if checkpoint else None)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bks46 (bks46-new-jersey-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
500,0.920100
1000,0.756400


TrainOutput(global_step=1000, training_loss=0.8382781677246094, metrics={'train_runtime': 770.5404, 'train_samples_per_second': 5.191, 'train_steps_per_second': 1.298, 'total_flos': 7435318198272000.0, 'train_loss': 0.8382781677246094, 'epoch': 2.0})

In [23]:
model.save_pretrained("qlora-lora-only")  #  this saves adapter_config.json + adapter_model.bin


In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

base_model = "tiiuae/falcon-rw-1b"  # or whatever you used
peft_model_path = "./qlora-finetuned-model"  # your output_dir


In [25]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [26]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",  # or "bfloat16" if supported
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

In [29]:
# Load model with LoRA adapter
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map="auto")
model = PeftModel.from_pretrained(model, "qlora-lora-only")
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(50304, 2048)
        (h): ModuleList(
          (0-23): 24 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=6144, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

In [32]:
# Inference
prompt = "Explain why the sky is blue."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

output = model.generate(input_ids, max_new_tokens=100, do_sample=True)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [33]:
print("\n=== Generated Response ===\n")
print(decoded.replace("\\n", "\n"))
print("\n==========================\n")


=== Generated Response ===

Explain why the sky is blue.
Blue sky is the natural color of the sky. The color depends on the relative humidity conditions and can change with atmospheric changes. The air pressure varies between different places in the atmosphere also the temperature of the atmosphere varies. So the color changes with different conditions.- News
- Sports
- Business
- Entertainment
- Community
- Lifestyles
- Opinion
- Driveway
- National
-
Kokanee falls in NPHL finals
The Kitsilano


